In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm, zscore
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import *
from sklearn.metrics.pairwise import sigmoid_kernel
from sklearn.metrics import log_loss
import itertools
import thutil

np.random.seed(1)

# V2U = ['Coach Tourney Appearances',
# 		'FG%',
# 		'Adj Off Efficiency',
# 		'Adj Def Efficiency',
# 		'Strength of Schedule',
# 		'Win %',
# 		'Major Conference',
# 		'Turnovers per game',
# 		'Wins Last 10 Games',
# 		'Coach Record',
# 		'Seed',
# 		'Won a Major Conference',
# 		'Rebounds']

V2U =  ['Seed',
'Wins Last 10 Games',
'Adj Def Efficiency',
'Adj Off Efficiency',
'Turnovers per game']

# Load all the game data
game_data = []
for year in range(2014, 2018+1):
	year = f'{year - 1}-{year}'
	rep = pd.read_csv(f'{year}_combo.csv')
	rep.rename(columns=lambda x: ' '.join(x.split()), inplace=True)
	rep[rep.select_dtypes(include=[np.number]).columns].apply(zscore)
	rep['pdiff'] = rep['Winner Points'] - rep['Loser points']
	rep['Winner'] = rep['Winner'] + '_' + year
	rep['Loser'] = rep['Loser'] + '_' + year
	game_data.append(rep)
game_data = pd.concat(game_data)

raw_names = [foo.strip() for foo in list(pd.read_csv('2014-2015_data.csv').columns)]
raw_names.remove('Team Name')

for i, row in game_data.iterrows():
	if np.random.random() > 0.5:
		for name in raw_names:
			vname_w = name + ' Winner'
			vname_l = name + ' Loser'
			wv = game_data.at[i, vname_w]
			game_data.at[i, vname_w] = game_data.at[i, vname_l]
			game_data.at[i, vname_l] = wv
		game_data.at[i, 'pdiff'] *= -1

def go(v2u, fi, sig1, sig2, ls1, ls2, ws, wl):
	# Set up winner, loser names for the selection of vars
	v2u_mod = []
	for v in v2u:
		v2u_mod.append(v + ' Winner')
		v2u_mod.append(v + ' Loser')

	# Set up Gaussian Process with its kernel (don't train yet)
	rbf1 = sig1 ** 2 * RBF(length_scale=ls1)
	rbf2 = sig2 ** 2 * RBF(length_scale=ls2)
	white = ws ** 2 * WhiteKernel(noise_level=wl**2)
	kernel = rbf1 + rbf2 + white

	gp = GaussianProcessRegressor(kernel,
# 		n_restarts_optimizer=10,
		normalize_y=True)

	lls = []
	for val_year in range(2014, 2018+1):
		val_year = f'{val_year - 1}-{val_year}'
		train = game_data[game_data['Winner'].str.endswith(val_year) == False]
		val = game_data[game_data['Winner'].str.endswith(val_year) == True]

		# Select the proper subsets of the dataframe
		X_train = train[v2u_mod]
		y_train = train['pdiff']
		X_val = val[v2u_mod]
		y_val = val['pdiff']

		gp.fit(X_train, y_train)	# train
		gp_y_pred, gp_sd = gp.predict(X_val, return_std=True)	# validate


		# Calculate validation log loss
		y_pred_probs = 1 - norm.cdf(-gp_y_pred / gp_sd)
		ll = log_loss(np.array(y_val > 0, dtype=np.int),
						np.array([1 - y_pred_probs, y_pred_probs]).T,
						labels=[0, 1])
		print(val_year + ': ' + str(ll))
		lls.append(ll)

# 	print(str(np.mean(lls)) + ', ' + str(v2u))
# 	fi.write(f'{np.mean(lls)}, {v2u}\n')

	return np.mean(lls)

sig1 = 3.2
sig2 = 3.2
ls1 = 93
ls2 = 93
ws = 1
wl = 1

# with open('ALOG.csv', 'a+') as fi:
# 	tp = thutil.ThreadPool(8)
# 	for nvars in range(3, 5+1):
# 	#for nvars in range(1, 3):
# 		for v2u in list(map(set, itertools.combinations(V2U, nvars))):
# 			tp.add_task(go, *(v2u, fi, sig1, sig2, ls1, ls2, ws, wl))
# 	tp.wait_completion()

# with open('tune.csv', 'a+') as fi:
# go(V2U, None, sig1, sig2, ls1, ls2, ws, wl)

In [2]:
best_ws = None
best_wl = None
best_ll = 1
for ws in np.linspace(0, 5, 20):
    print('x')
    for wl in np.linspace(0, 5, 20):
        ll = go(V2U, None, sig1, sig2, ls1, ls2, ws, wl)
        if ll < best_ll:
            best_ll = ll
            best_ws = ws
            best_wl = wl

print(f'Best WS: {best_ws}')
print(f'Best WL: {best_wl}')

x


/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/ke

/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/gp

/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ 3.10493699e-05, -0.00000000e+00,  3.10493699e-05, -0.00000000e+00,
        2.50066610e-01,  2.50066610e-01]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 67, 'nit': 16, 'warnflag': 2}
  ConvergenceWarning)
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-

x


...

x


/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/gp

x


/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ 0.00050238, -0.        ,  0.00050238, -0.        ,  0.25006661,
        0.25006661]

Best WS: 1.8421052631578947
Best WL: 2.3684210526315788


In [3]:
best_ll

0.5149891016544332

In [4]:
best_ls1m = None
best_ls2m = None

for ls1m in np.linspace(0, 2, 20):
    print('x')
    for ls2m in np.linspace(0, 2, 20):
        ll = go(V2U, None, sig1, sig2, ls1 * ls1m, ls2 * ls2m, best_ws, best_wl)
        if ll < best_ll:
            best_ll = ll
            best_ls1m = ls1m
            best_ls2m = ls2m


print(f'Best WS: {best_ws}')
print(f'Best WL: {best_wl}')

x


/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ 3.76675183e-05, -0.00000000e+00,  3.76675183e-05, -0.00000000e+00,
        2.50066610e-01,  2.50066610e-01]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 71, 'nit': 21, 'warnflag': 2}
  ConvergenceWarning)
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-

/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.05701226, -0.        , -0.01629556, -0.01824846,  0.2501644 ,
        0.2501644 ]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 97, 'nit': 35, 'warnflag': 2}
  ConvergenceWarning)
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussia

/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ 0.0018744 , -0.        , -0.00207295, -0.00194088,  0.25033349,
        0.25033349]

/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ 0.01020176, -0.        ,  0.00438102, -0.01098374,  0.25014661,
        0.25014661]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 98, 'nit': 31, 'warnflag': 2}
  ConvergenceWarning)
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00926398, -0.        ,  0.00298657, -0.00476788,  0.25013772,
       

...

x


/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/gp

x


/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00296955,  0.00655731,  0.03959247, -0.        ,  0.25031568,
        0.25031568]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 129, 'nit': 38, 'warnflag': 2}
  ConvergenceWarning)
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussian_process/kernels.py:247: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
/home/rytse/anaconda3/envs/mm/lib/python3.6/site-packages/sklearn/gaussi

Best WS: 1.8421052631578947
Best WL: 2.3684210526315788


In [28]:
print(f'Best LS1M: {best_ls1m}')
print(f'Best LS2M: {best_ls2m}')

Best LS1M: 0.631578947368421
Best LS2M: 0.3157894736842105


In [12]:
def go(v2u, fi, sig1, sig2, ls1, ls2, ws, wl):
	# Set up winner, loser names for the selection of vars
	v2u_mod = []
	for v in v2u:
		v2u_mod.append(v + ' Winner')
		v2u_mod.append(v + ' Loser')

	# Set up Gaussian Process with its kernel (don't train yet)
	rbf1 = sig1 ** 2 * RBF(length_scale=ls1)
	rbf2 = sig2 ** 2 * RBF(length_scale=ls2)
	white = ws ** 2 * WhiteKernel(noise_level=wl**2)
	kernel = rbf1 + rbf2 + white

	gp = GaussianProcessRegressor(kernel,
# 		n_restarts_optimizer=10,
		normalize_y=True)

	lls = []
	for val_year in [2014, 2015, 2017]:#range(2014, 2018+1):
		val_year = f'{val_year - 1}-{val_year}'
		train = game_data[game_data['Winner'].str.endswith(val_year) == False]
		val = game_data[game_data['Winner'].str.endswith(val_year) == True]

		# Select the proper subsets of the dataframe
		X_train = train[v2u_mod]
		y_train = train['pdiff']
		X_val = val[v2u_mod]
		y_val = val['pdiff']

		gp.fit(X_train, y_train)	# train
		gp_y_pred, gp_sd = gp.predict(X_val, return_std=True)	# validate


		# Calculate validation log loss
		y_pred_probs = 1 - norm.cdf(-gp_y_pred / gp_sd)
		ll = log_loss(np.array(y_val > 0, dtype=np.int),
						np.array([1 - y_pred_probs, y_pred_probs]).T,
						labels=[0, 1])
		print(val_year + ': ' + str(ll))
		lls.append(ll)

# 	print(str(np.mean(lls)) + ', ' + str(v2u))
# 	fi.write(f'{np.mean(lls)}, {v2u}\n')

	return np.mean(lls)


In [13]:
go(V2U, None, sig1, sig2, ls1 * best_ls1m, ls2 * best_ls2m, best_ws, best_wl)

2013-2014: 0.5182266544313968
2014-2015: 0.4495148746180374
2016-2017: 0.41701733448107425


0.46158628784350286

In [29]:
v2u_mod = []
for v in V2U:
    v2u_mod.append(v + ' Winner')
    v2u_mod.append(v + ' Loser')

train = game_data[game_data['Winner'].str.endswith('2014') == False]
train = train[train['Winner'].str.endswith('2015') == False]
train = train[train['Winner'].str.endswith('2017') == False]
val = game_data[game_data['Winner'].str.endswith('2016') == False]
val = val[val['Winner'].str.endswith('2018') == False]

# Select the proper subsets of the dataframe
X_train = train[v2u_mod]
y_train = train['pdiff']
X_val = val[v2u_mod]
y_val = val['pdiff']

rbf1 = sig1 ** 2 * RBF(length_scale=ls1 * ls1m)
rbf2 = sig2 ** 2 * RBF(length_scale=ls2 * ls2m)
white = ws ** 2 * WhiteKernel(noise_level=wl**2)
kernel = rbf1 + rbf2 + white

gp = GaussianProcessRegressor(kernel,
# 		n_restarts_optimizer=10,
    normalize_y=True)

gp.fit(X_train, y_train)    # train
gp_y_pred, gp_sd = gp.predict(X_val, return_std=True)    # validate

# Calculate validation log loss
y_pred_probs = 1 - norm.cdf(-gp_y_pred / gp_sd)
ll = log_loss(np.array(y_val > 0, dtype=np.int),
            np.array([1 - y_pred_probs, y_pred_probs]).T,
            labels=[0, 1])
print(str(ll))

0.5079430820607647


In [30]:
v2u_mod = []
for v in V2U:
    v2u_mod.append(v + ' Winner')
    v2u_mod.append(v + ' Loser')

train = game_data#[game_data['Winner'].str.endswith('2016') == False]
# train = train[train['Winner'].str.endswith('2018') == False]
val = game_data[game_data['Winner'].str.endswith('2014') == False]
val = val[val['Winner'].str.endswith('2015') == False]
val = val[val['Winner'].str.endswith('2017') == False]

# Select the proper subsets of the dataframe
X_train = train[v2u_mod]
y_train = train['pdiff']
X_val = val[v2u_mod]
y_val = val['pdiff']

rbf1 = sig1 ** 2 * RBF(length_scale=ls1*ls1m)
rbf2 = sig2 ** 2 * RBF(length_scale=ls2*ls2m)
white = ws ** 2 * WhiteKernel(noise_level=wl**2)
kernel = rbf1 + rbf2 + white

gp = GaussianProcessRegressor(kernel,
# 		n_restarts_optimizer=10,
    normalize_y=True)

gp.fit(X_train, y_train)    # train
gp_y_pred, gp_sd = gp.predict(X_val, return_std=True)    # validate

# Calculate validation log loss
y_pred_probs = 1 - norm.cdf(-gp_y_pred / gp_sd)
ll = log_loss(np.array(y_val > 0, dtype=np.int),
            np.array([1 - y_pred_probs, y_pred_probs]).T,
            labels=[0, 1])
print(str(ll))

0.48021433665329283
